# Tuning and Optimizing Neural Networks - Lab

## Introduction

Now that you've practiced regularization, initialization, and optimization techniques, its time to synthesize these concepts into a cohesive modeling pipeline.  

With this pipeline, you will not only fit an initial model but also attempt to improve it. Your final model selection will pertain to the test metrics across these models. This will more naturally simulate a problem you might be faced with in practice, and the various modeling decisions you are apt to encounter along the way.  

Recall that our end objective is to achieve a balance between overfitting and underfitting. You've seen the bias variance trade-off, and the role of regularization in order to reduce overfitting on training data and improving generalization to new cases. Common frameworks for such a procedure include train/validate/test methodology when data is plentiful, and K-folds cross-validation for smaller, more limited datasets. In this lab, you'll perform the latter, as the dataset in question is fairly limited. 

## Objectives

You will be able to:

* Apply normalization as a preprocessing technique 
* Implement a K-folds cross validation modeling pipeline for deep learning models 
* Apply regularization techniques to improve your model's performance 

## Load the data

First, run the following cell to import all the neccessary libraries and classes you will need in this lab. 

In [1]:
# Necessary libraries and classes
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_predict
from keras import models
from keras import layers
from keras import regularizers
from keras.wrappers.scikit_learn import KerasRegressor

In this lab you'll be working with the *The Lending Club* data. 

- Import the data available in the file `'loan_final.csv'` 
- Drop rows with missing values in the `'total_pymnt'` column (this is your target column) 
- Print the first five rows of the data 
- Print the dimensions of the data 

In [5]:
# Import the data
data = pd.read_csv('loan_final.csv')

# Drop rows with no target value
data.dropna(subset=['total_pymnt'], inplace=True)

# Print the first five rows
data.head()

,loan_amnt,funded_amnt_inv,term,int_rate,installment,grade,emp_length,home_ownership,annual_inc,verification_status,loan_status,purpose,addr_state,total_acc,total_pymnt,application_type
0,5000.0,4975.0,36 months,10.65%,162.87,B,10+ years,RENT,24000.0,Verified,Fully Paid,credit_card,AZ,9.0,5863.155187,Individual
1,2500.0,2500.0,60 months,15.27%,59.83,C,< 1 year,RENT,30000.0,Source Verified,Charged Off,car,GA,4.0,1014.530000,Individual
2,2400.0,2400.0,36 months,15.96%,84.33,C,10+ years,RENT,12252.0,Not Verified,Fully Paid,small_business,IL,10.0,3005.666844,Individual
3,10000.0,10000.0,36 months,13.49%,339.31,C,10+ years,RENT,49200.0,Source Verified,Fully Paid,other,CA,37.0,12231.890000,Individual
4,3000.0,3000.0,60 months,12.69%,67.79,B,1 year,RENT,80000.0,Source Verified,Fully Paid,other,OR,38.0,4066.908161,Individual


In [6]:
# Print the dimensions of data 
data.shape

(42535, 16)

## Generating a Hold Out Test Set

While we will be using K-fold cross validation to select an optimal model, we still want a final hold out test set that is completely independent of any modeling decisions. As such, pull out a sample of 30% of the total available data. For consistency of results, use random seed 42. 

In [8]:
# Features to build the model
features = ['loan_amnt', 'funded_amnt_inv', 'installment', 'annual_inc', 
            'home_ownership', 'verification_status', 'emp_length']

X = data[features]
y = data[['total_pymnt']]

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

## Preprocessing (Numerical features) 

- Fill all missing values in numeric features with their respective means 
- Standardize all the numeric features  
- Convert the final results into DataFrames 

In [41]:
# Select continuous features
cont_features = ['loan_amnt', 'funded_amnt_inv', 'installment', 'annual_inc']

X_train_cont = X_train.loc[:, cont_features]
X_test_cont = X_test.loc[:, cont_features]

# Instantiate SimpleImputer - fill the missing values with the mean
si = SimpleImputer()

# Fit and transform the training data
X_train_imputed = si.fit_transform(X_train_cont)

# Transform test data
X_test_imputed = si.transform(X_test_cont)

# Instantiate StandardScaler
ss_X = StandardScaler()

# Fit and transform the training data
X_train_scaled = pd.DataFrame(ss_X.fit_transform(X_train_imputed), columns=cont_features)

# Transform test data
X_test_scaled = pd.DataFrame(ss_X.transform(X_test_imputed), columns=cont_features)

## Preprocessing (Categorical features) 

- Fill all missing values in categorical features with the string `'missing'` 
- One-hot encode all categorical features 
- Convert the final results into DataFrames 


In [42]:
# Select only the categorical features
cat_features = ['home_ownership', 'verification_status', 'emp_length']
X_train_cat = X_train.loc[:, cat_features]
X_test_cat = X_test.loc[:, cat_features]

# Fill missing values with the string 'missing'
X_train_cat.fillna('missing', inplace=True)
X_test_cat.fillna('missing', inplace=True)


# OneHotEncode categorical variables
ohe = OneHotEncoder()

# Transform training and test sets
X_train_ohe = ohe.fit_transform(X_train_cat)
X_test_ohe = ohe.transform(X_test_cat)

In [43]:
# Get all categorical feature names
cat_columns = ohe.get_feature_names(input_features=X_train_cat.columns)

# Fit and transform the training data
X_train_categorical = pd.DataFrame(X_train_ohe.todense(), columns=cat_columns)

# Transform test data
X_test_categorical = pd.DataFrame(X_test_ohe.todense(), columns=cat_columns)

In [44]:
X_train_categorical.head()

,home_ownership_MORTGAGE,home_ownership_NONE,home_ownership_OTHER,home_ownership_OWN,home_ownership_RENT,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,emp_length_1 year,emp_length_10+ years,emp_length_2 years,emp_length_3 years,emp_length_4 years,emp_length_5 years,emp_length_6 years,emp_length_7 years,emp_length_8 years,emp_length_9 years,emp_length_< 1 year,emp_length_missing
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Run the below cell to combine the numeric and categorical features. 

In [45]:
# Combine continuous and categorical feature DataFrames
X_train_all = pd.concat([X_train_scaled, X_train_categorical], axis=1)
X_test_all = pd.concat([X_test_scaled, X_test_categorical], axis=1)

# Number of input features
n_features = X_train_all.shape[1]

- Standardize the target DataFrames (`y_train` and `y_test`) 

In [46]:
# Instantiate StandardScaler
ss_y = StandardScaler()

# Fit and transform Y (train)
y_train_scaled = ss_y.fit_transform(y_train)

# Transform test Y (test)
y_test_scaled = ss_y.transform(y_test)

## Define a K-fold Cross Validation Methodology

Now that your have a complete holdout test set, you will perform k-fold cross-validation using the following steps: 

- Create a function that returns a compiled deep learning model 
- Use the wrapper function `KerasRegressor()` that defines how these folds are trained 
- Call the `cross_val_predict()` function to perform k-fold cross-validation 

In the cell below, we've defined a baseline model that returns a compiled Keras models. 

In [47]:
# Define a function that returns a compiled Keras model 
def create_baseline_model():
    
    # Initialize model
    model = models.Sequential()

    # First hidden layer
    model.add(layers.Dense(10, activation='relu', input_shape=(n_features,)))

    # Second hidden layer
    model.add(layers.Dense(5, activation='relu'))

    # Output layer
    model.add(layers.Dense(1, activation='linear'))

    # Compile the model
    model.compile(optimizer='SGD', 
                  loss='mse',  
                  metrics=['mse']) 
    
    # Return the compiled model
    return model

Wrap `create_baseline_model` inside a call to `KerasRegressor()`, and: 

- Train for 150 epochs 
- Set the batch size to 256 

> NOTE: Refer to the [documentation](https://keras.io/scikit-learn-api/) to learn about `KerasRegressor()`.  

In [48]:
# Wrap the above function for use in cross-validation
keras_wrapper_1 = KerasRegressor(create_baseline_model, epochs=150, batch_size=256)

Use `cross_val_predict()` to generate cross-validated predictions with: 
- 5-fold cv 
- scaled input (`X_train_all`) and output (`y_train_scaled`) 

In [49]:
# ⏰ This cell may take several mintes to run
# Generate cross-validated predictions
np.random.seed(123)
cv_baseline_preds = cross_val_predict(keras_wrapper_1, X_train_all, y_train_scaled, cv=5)

Epoch 1/150
94/94 [==============================] - 0s 635us/step - loss: 0.5620 - mse: 0.5620
Epoch 2/150
94/94 [==============================] - 0s 626us/step - loss: 0.2757 - mse: 0.2757
Epoch 3/150
94/94 [==============================] - 0s 691us/step - loss: 0.2519 - mse: 0.2519
Epoch 4/150
94/94 [==============================] - 0s 638us/step - loss: 0.2401 - mse: 0.2401
Epoch 5/150
94/94 [==============================] - 0s 613us/step - loss: 0.2325 - mse: 0.2325
Epoch 6/150
94/94 [==============================] - 0s 590us/step - loss: 0.2274 - mse: 0.2274
Epoch 7/150
94/94 [==============================] - 0s 613us/step - loss: 0.2235 - mse: 0.2235
Epoch 8/150
94/94 [==============================] - 0s 588us/step - loss: 0.2204 - mse: 0.2204
Epoch 9/150
94/94 [==============================] - 0s 591us/step - loss: 0.2181 - mse: 0.2181
Epoch 10/150
94/94 [==============================] - 0s 597us/step - loss: 0.2160 - mse: 0.2160
Epoch 11/150
94/94 [===================

94/94 [==============================] - 0s 602us/step - loss: 0.1966 - mse: 0.1966
Epoch 86/150
94/94 [==============================] - 0s 592us/step - loss: 0.1965 - mse: 0.1965
Epoch 87/150
94/94 [==============================] - 0s 607us/step - loss: 0.1963 - mse: 0.1963
Epoch 88/150
94/94 [==============================] - 0s 588us/step - loss: 0.1963 - mse: 0.1963
Epoch 89/150
94/94 [==============================] - 0s 558us/step - loss: 0.1961 - mse: 0.1961
Epoch 90/150
94/94 [==============================] - 0s 639us/step - loss: 0.1962 - mse: 0.1962
Epoch 91/150
94/94 [==============================] - 0s 603us/step - loss: 0.1961 - mse: 0.1961
Epoch 92/150
94/94 [==============================] - 0s 612us/step - loss: 0.1962 - mse: 0.1962
Epoch 93/150
94/94 [==============================] - 0s 581us/step - loss: 0.1959 - mse: 0.1959
Epoch 94/150
94/94 [==============================] - 0s 615us/step - loss: 0.1958 - mse: 0.1958
Epoch 95/150
94/94 [=======================

94/94 [==============================] - 0s 617us/step - loss: 0.2457 - mse: 0.2457
Epoch 20/150
94/94 [==============================] - 0s 596us/step - loss: 0.2430 - mse: 0.2430
Epoch 21/150
94/94 [==============================] - 0s 577us/step - loss: 0.2404 - mse: 0.2404
Epoch 22/150
94/94 [==============================] - 0s 589us/step - loss: 0.2382 - mse: 0.2382
Epoch 23/150
94/94 [==============================] - 0s 598us/step - loss: 0.2360 - mse: 0.2360
Epoch 24/150
94/94 [==============================] - 0s 646us/step - loss: 0.2339 - mse: 0.2339
Epoch 25/150
94/94 [==============================] - 0s 657us/step - loss: 0.2321 - mse: 0.2321
Epoch 26/150
94/94 [==============================] - 0s 618us/step - loss: 0.2304 - mse: 0.2304
Epoch 27/150
94/94 [==============================] - 0s 601us/step - loss: 0.2289 - mse: 0.2289
Epoch 28/150
94/94 [==============================] - 0s 614us/step - loss: 0.2275 - mse: 0.2275
Epoch 29/150
94/94 [=======================

94/94 [==============================] - 0s 617us/step - loss: 0.2022 - mse: 0.2022
Epoch 104/150
94/94 [==============================] - 0s 593us/step - loss: 0.2022 - mse: 0.2022
Epoch 105/150
94/94 [==============================] - 0s 610us/step - loss: 0.2021 - mse: 0.2021
Epoch 106/150
94/94 [==============================] - 0s 553us/step - loss: 0.2020 - mse: 0.2020
Epoch 107/150
94/94 [==============================] - 0s 594us/step - loss: 0.2019 - mse: 0.2019
Epoch 108/150
94/94 [==============================] - 0s 620us/step - loss: 0.2017 - mse: 0.2017
Epoch 109/150
94/94 [==============================] - 0s 579us/step - loss: 0.2017 - mse: 0.2017
Epoch 110/150
94/94 [==============================] - 0s 640us/step - loss: 0.2017 - mse: 0.2017
Epoch 111/150
94/94 [==============================] - 0s 633us/step - loss: 0.2017 - mse: 0.2017
Epoch 112/150
94/94 [==============================] - 0s 625us/step - loss: 0.2016 - mse: 0.2016
Epoch 113/150
94/94 [=============

94/94 [==============================] - 0s 634us/step - loss: 0.2005 - mse: 0.2005
Epoch 38/150
94/94 [==============================] - 0s 656us/step - loss: 0.2002 - mse: 0.2002
Epoch 39/150
94/94 [==============================] - 0s 600us/step - loss: 0.2001 - mse: 0.2001
Epoch 40/150
94/94 [==============================] - 0s 598us/step - loss: 0.2001 - mse: 0.2001
Epoch 41/150
94/94 [==============================] - 0s 621us/step - loss: 0.2000 - mse: 0.2000
Epoch 42/150
94/94 [==============================] - 0s 548us/step - loss: 0.1998 - mse: 0.1998
Epoch 43/150
94/94 [==============================] - 0s 599us/step - loss: 0.1997 - mse: 0.1997
Epoch 44/150
94/94 [==============================] - 0s 565us/step - loss: 0.1998 - mse: 0.1998
Epoch 45/150
94/94 [==============================] - 0s 581us/step - loss: 0.1998 - mse: 0.1998
Epoch 46/150
94/94 [==============================] - 0s 551us/step - loss: 0.1993 - mse: 0.1993
Epoch 47/150
94/94 [=======================

94/94 [==============================] - 0s 621us/step - loss: 0.1923 - mse: 0.1923
Epoch 122/150
94/94 [==============================] - 0s 613us/step - loss: 0.1922 - mse: 0.1922
Epoch 123/150
94/94 [==============================] - 0s 612us/step - loss: 0.1921 - mse: 0.1921
Epoch 124/150
94/94 [==============================] - 0s 632us/step - loss: 0.1921 - mse: 0.1921
Epoch 125/150
94/94 [==============================] - 0s 608us/step - loss: 0.1920 - mse: 0.1920
Epoch 126/150
94/94 [==============================] - 0s 604us/step - loss: 0.1919 - mse: 0.1919
Epoch 127/150
94/94 [==============================] - 0s 655us/step - loss: 0.1918 - mse: 0.1918
Epoch 128/150
94/94 [==============================] - 0s 620us/step - loss: 0.1918 - mse: 0.1918
Epoch 129/150
94/94 [==============================] - 0s 619us/step - loss: 0.1917 - mse: 0.1917
Epoch 130/150
94/94 [==============================] - 0s 634us/step - loss: 0.1916 - mse: 0.1916
Epoch 131/150
94/94 [=============

94/94 [==============================] - 0s 650us/step - loss: 0.2026 - mse: 0.2026
Epoch 56/150
94/94 [==============================] - 0s 639us/step - loss: 0.2025 - mse: 0.2025
Epoch 57/150
94/94 [==============================] - 0s 654us/step - loss: 0.2024 - mse: 0.2024
Epoch 58/150
94/94 [==============================] - 0s 683us/step - loss: 0.2022 - mse: 0.2022
Epoch 59/150
94/94 [==============================] - 0s 616us/step - loss: 0.2020 - mse: 0.2020
Epoch 60/150
94/94 [==============================] - 0s 579us/step - loss: 0.2019 - mse: 0.2019
Epoch 61/150
94/94 [==============================] - 0s 612us/step - loss: 0.2018 - mse: 0.2018
Epoch 62/150
94/94 [==============================] - 0s 632us/step - loss: 0.2017 - mse: 0.2017
Epoch 63/150
94/94 [==============================] - 0s 624us/step - loss: 0.2017 - mse: 0.2017
Epoch 64/150
94/94 [==============================] - 0s 604us/step - loss: 0.2013 - mse: 0.2013
Epoch 65/150
94/94 [=======================

94/94 [==============================] - 0s 603us/step - loss: 0.1946 - mse: 0.1946
Epoch 140/150
94/94 [==============================] - 0s 557us/step - loss: 0.1945 - mse: 0.1945
Epoch 141/150
94/94 [==============================] - 0s 563us/step - loss: 0.1945 - mse: 0.1945
Epoch 142/150
94/94 [==============================] - 0s 602us/step - loss: 0.1944 - mse: 0.1944
Epoch 143/150
94/94 [==============================] - 0s 708us/step - loss: 0.1944 - mse: 0.1944
Epoch 144/150
94/94 [==============================] - 0s 643us/step - loss: 0.1942 - mse: 0.1942
Epoch 145/150
94/94 [==============================] - 0s 679us/step - loss: 0.1941 - mse: 0.1941
Epoch 146/150
94/94 [==============================] - 0s 614us/step - loss: 0.1942 - mse: 0.1942
Epoch 147/150
94/94 [==============================] - 0s 584us/step - loss: 0.1941 - mse: 0.1941
Epoch 148/150
94/94 [==============================] - 0s 559us/step - loss: 0.1940 - mse: 0.1940
Epoch 149/150
94/94 [=============

94/94 [==============================] - 0s 597us/step - loss: 0.1961 - mse: 0.1961
Epoch 74/150
94/94 [==============================] - 0s 589us/step - loss: 0.1961 - mse: 0.1961
Epoch 75/150
94/94 [==============================] - 0s 616us/step - loss: 0.1959 - mse: 0.1959
Epoch 76/150
94/94 [==============================] - 0s 586us/step - loss: 0.1958 - mse: 0.1958
Epoch 77/150
94/94 [==============================] - 0s 557us/step - loss: 0.1957 - mse: 0.1957
Epoch 78/150
94/94 [==============================] - 0s 579us/step - loss: 0.1955 - mse: 0.1955
Epoch 79/150
94/94 [==============================] - 0s 530us/step - loss: 0.1956 - mse: 0.1956
Epoch 80/150
94/94 [==============================] - 0s 596us/step - loss: 0.1954 - mse: 0.1954
Epoch 81/150
94/94 [==============================] - 0s 543us/step - loss: 0.1953 - mse: 0.1953
Epoch 82/150
94/94 [==============================] - 0s 556us/step - loss: 0.1952 - mse: 0.1952
Epoch 83/150
94/94 [=======================

- Find the RMSE on train data 

In [50]:
cv_baseline_preds

array([ 0.48704448,  0.54387605, -0.35010782, ..., -0.89773756,
       -0.45397437,  0.38764614], dtype=float32)

In [53]:
# RMSE on train data (scaled)
mean_squared_error(y_train_scaled, cv_baseline_preds, squared=False)

0.44360937157105107

- Convert the scaled predictions back to original scale 
- Calculate RMSE in the original units with `y_train` and `baseline_preds` 

In [54]:
# Convert the predictions back to original scale
baseline_preds = ss_y.inverse_transform(cv_baseline_preds)

# RMSE on train data (original scale)
mean_squared_error(y_train, baseline_preds, squared=False)

4040.633499384675

## Intentionally Overfitting a Model

Now that you've developed a baseline model, its time to intentionally overfit a model. To overfit a model, you can:
* Add layers
* Make the layers bigger
* Increase the number of training epochs

Again, be careful here. Think about the limitations of your resources, both in terms of your computers specs and how much time and patience you have to let the process run. Also keep in mind that you will then be regularizing these overfit models, meaning another round of experiments and more time and resources.

In [59]:
# Define a function that returns a compiled Keras model 
def create_bigger_model():
        
    # Initialize model
    model = models.Sequential()

    # First hidden layer
    model.add(layers.Dense(100, activation='relu')) #, input_shape=(n_features,)))

    # Second hidden layer
    model.add(layers.Dense(50, activation='relu'))

    model.add(layers.Dense(25, activation='relu'))

    # Output layer
    model.add(layers.Dense(1, activation='linear'))

    # Compile the model
    model.compile(optimizer='SGD', 
                  loss='mse',  
                  metrics=['mse']) 
    
    # Return the compiled model
    return model

In [63]:
# Wrap the above function for use in cross-validation
keras_wrapper_2 = KerasRegressor(create_bigger_model, epochs=200, batch_size=256)

In [66]:
# ⏰ This cell may take several mintes to run
# Generate cross-validated predictions
np.random.seed(123)
cv_bigger_model_preds = cross_val_predict(keras_wrapper_1, X_train_all, y_train_scaled, cv=5)

Epoch 1/150
94/94 [==============================] - 0s 628us/step - loss: 0.4398 - mse: 0.4398
Epoch 2/150
94/94 [==============================] - 0s 566us/step - loss: 0.2725 - mse: 0.2725
Epoch 3/150
94/94 [==============================] - 0s 564us/step - loss: 0.2517 - mse: 0.2517
Epoch 4/150
94/94 [==============================] - 0s 578us/step - loss: 0.2409 - mse: 0.2409
Epoch 5/150
94/94 [==============================] - 0s 562us/step - loss: 0.2335 - mse: 0.2335
Epoch 6/150
94/94 [==============================] - 0s 629us/step - loss: 0.2289 - mse: 0.2289
Epoch 7/150
94/94 [==============================] - 0s 601us/step - loss: 0.2246 - mse: 0.2246
Epoch 8/150
94/94 [==============================] - 0s 612us/step - loss: 0.2220 - mse: 0.2220
Epoch 9/150
94/94 [==============================] - 0s 623us/step - loss: 0.2200 - mse: 0.2200
Epoch 10/150
94/94 [==============================] - 0s 583us/step - loss: 0.2178 - mse: 0.2178
Epoch 11/150
94/94 [===================

94/94 [==============================] - 0s 582us/step - loss: 0.1969 - mse: 0.1969
Epoch 86/150
94/94 [==============================] - 0s 585us/step - loss: 0.1966 - mse: 0.1966
Epoch 87/150
94/94 [==============================] - 0s 591us/step - loss: 0.1965 - mse: 0.1965
Epoch 88/150
94/94 [==============================] - 0s 569us/step - loss: 0.1964 - mse: 0.1964
Epoch 89/150
94/94 [==============================] - 0s 574us/step - loss: 0.1964 - mse: 0.1964
Epoch 90/150
94/94 [==============================] - 0s 592us/step - loss: 0.1962 - mse: 0.1962
Epoch 91/150
94/94 [==============================] - 0s 667us/step - loss: 0.1961 - mse: 0.1961
Epoch 92/150
94/94 [==============================] - 0s 624us/step - loss: 0.1960 - mse: 0.1960
Epoch 93/150
94/94 [==============================] - 0s 606us/step - loss: 0.1959 - mse: 0.1959
Epoch 94/150
94/94 [==============================] - 0s 599us/step - loss: 0.1959 - mse: 0.1959
Epoch 95/150
94/94 [=======================

94/94 [==============================] - 0s 571us/step - loss: 0.2120 - mse: 0.2120
Epoch 20/150
94/94 [==============================] - 0s 568us/step - loss: 0.2114 - mse: 0.2114
Epoch 21/150
94/94 [==============================] - 0s 601us/step - loss: 0.2111 - mse: 0.2111
Epoch 22/150
94/94 [==============================] - 0s 588us/step - loss: 0.2103 - mse: 0.2103
Epoch 23/150
94/94 [==============================] - 0s 592us/step - loss: 0.2100 - mse: 0.2100
Epoch 24/150
94/94 [==============================] - 0s 631us/step - loss: 0.2095 - mse: 0.2095
Epoch 25/150
94/94 [==============================] - 0s 570us/step - loss: 0.2090 - mse: 0.2090
Epoch 26/150
94/94 [==============================] - 0s 598us/step - loss: 0.2088 - mse: 0.2088
Epoch 27/150
94/94 [==============================] - 0s 632us/step - loss: 0.2084 - mse: 0.2084
Epoch 28/150
94/94 [==============================] - 0s 674us/step - loss: 0.2081 - mse: 0.2081
Epoch 29/150
94/94 [=======================

94/94 [==============================] - 0s 602us/step - loss: 0.1968 - mse: 0.1968
Epoch 104/150
94/94 [==============================] - 0s 568us/step - loss: 0.1965 - mse: 0.1965
Epoch 105/150
94/94 [==============================] - 0s 624us/step - loss: 0.1966 - mse: 0.1966
Epoch 106/150
94/94 [==============================] - 0s 642us/step - loss: 0.1964 - mse: 0.1964
Epoch 107/150
94/94 [==============================] - 0s 575us/step - loss: 0.1964 - mse: 0.1964
Epoch 108/150
94/94 [==============================] - 0s 595us/step - loss: 0.1965 - mse: 0.1965
Epoch 109/150
94/94 [==============================] - 0s 601us/step - loss: 0.1964 - mse: 0.1964
Epoch 110/150
94/94 [==============================] - 0s 584us/step - loss: 0.1964 - mse: 0.1964
Epoch 111/150
94/94 [==============================] - 0s 578us/step - loss: 0.1964 - mse: 0.1964
Epoch 112/150
94/94 [==============================] - 0s 569us/step - loss: 0.1961 - mse: 0.1961
Epoch 113/150
94/94 [=============

94/94 [==============================] - 0s 615us/step - loss: 0.2121 - mse: 0.2121
Epoch 38/150
94/94 [==============================] - 0s 642us/step - loss: 0.2113 - mse: 0.2113
Epoch 39/150
94/94 [==============================] - 0s 611us/step - loss: 0.2106 - mse: 0.2106
Epoch 40/150
94/94 [==============================] - 0s 614us/step - loss: 0.2099 - mse: 0.2099
Epoch 41/150
94/94 [==============================] - 0s 653us/step - loss: 0.2092 - mse: 0.2092
Epoch 42/150
94/94 [==============================] - 0s 635us/step - loss: 0.2085 - mse: 0.2085
Epoch 43/150
94/94 [==============================] - 0s 626us/step - loss: 0.2079 - mse: 0.2079
Epoch 44/150
94/94 [==============================] - 0s 629us/step - loss: 0.2072 - mse: 0.2072
Epoch 45/150
94/94 [==============================] - 0s 568us/step - loss: 0.2066 - mse: 0.2066
Epoch 46/150
94/94 [==============================] - 0s 604us/step - loss: 0.2062 - mse: 0.2062
Epoch 47/150
94/94 [=======================

94/94 [==============================] - 0s 628us/step - loss: 0.1919 - mse: 0.1919
Epoch 122/150
94/94 [==============================] - 0s 593us/step - loss: 0.1920 - mse: 0.1920
Epoch 123/150
94/94 [==============================] - 0s 594us/step - loss: 0.1917 - mse: 0.1917
Epoch 124/150
94/94 [==============================] - 0s 645us/step - loss: 0.1916 - mse: 0.1916
Epoch 125/150
94/94 [==============================] - 0s 612us/step - loss: 0.1916 - mse: 0.1916
Epoch 126/150
94/94 [==============================] - 0s 571us/step - loss: 0.1915 - mse: 0.1915
Epoch 127/150
94/94 [==============================] - 0s 606us/step - loss: 0.1915 - mse: 0.1915
Epoch 128/150
94/94 [==============================] - 0s 564us/step - loss: 0.1915 - mse: 0.1915
Epoch 129/150
94/94 [==============================] - 0s 598us/step - loss: 0.1913 - mse: 0.1913
Epoch 130/150
94/94 [==============================] - 0s 612us/step - loss: 0.1911 - mse: 0.1911
Epoch 131/150
94/94 [=============

94/94 [==============================] - 0s 701us/step - loss: 0.1978 - mse: 0.1978
Epoch 56/150
94/94 [==============================] - 0s 636us/step - loss: 0.1979 - mse: 0.1979
Epoch 57/150
94/94 [==============================] - 0s 616us/step - loss: 0.1977 - mse: 0.1977
Epoch 58/150
94/94 [==============================] - 0s 642us/step - loss: 0.1974 - mse: 0.1974
Epoch 59/150
94/94 [==============================] - 0s 630us/step - loss: 0.1975 - mse: 0.1975
Epoch 60/150
94/94 [==============================] - 0s 571us/step - loss: 0.1973 - mse: 0.1973
Epoch 61/150
94/94 [==============================] - 0s 600us/step - loss: 0.1972 - mse: 0.1972
Epoch 62/150
94/94 [==============================] - 0s 620us/step - loss: 0.1971 - mse: 0.1971
Epoch 63/150
94/94 [==============================] - 0s 579us/step - loss: 0.1968 - mse: 0.1968
Epoch 64/150
94/94 [==============================] - 0s 647us/step - loss: 0.1968 - mse: 0.1968
Epoch 65/150
94/94 [=======================

94/94 [==============================] - 0s 579us/step - loss: 0.1918 - mse: 0.1918
Epoch 140/150
94/94 [==============================] - 0s 615us/step - loss: 0.1917 - mse: 0.1917
Epoch 141/150
94/94 [==============================] - 0s 652us/step - loss: 0.1918 - mse: 0.1918
Epoch 142/150
94/94 [==============================] - 0s 572us/step - loss: 0.1916 - mse: 0.1916
Epoch 143/150
94/94 [==============================] - 0s 560us/step - loss: 0.1918 - mse: 0.1918
Epoch 144/150
94/94 [==============================] - 0s 580us/step - loss: 0.1917 - mse: 0.1917
Epoch 145/150
94/94 [==============================] - 0s 583us/step - loss: 0.1916 - mse: 0.1916
Epoch 146/150
94/94 [==============================] - 0s 624us/step - loss: 0.1917 - mse: 0.1917
Epoch 147/150
94/94 [==============================] - 0s 547us/step - loss: 0.1916 - mse: 0.1916
Epoch 148/150
94/94 [==============================] - 0s 583us/step - loss: 0.1915 - mse: 0.1915
Epoch 149/150
94/94 [=============

94/94 [==============================] - 0s 614us/step - loss: 0.1971 - mse: 0.1971
Epoch 74/150
94/94 [==============================] - 0s 610us/step - loss: 0.1970 - mse: 0.1970
Epoch 75/150
94/94 [==============================] - 0s 622us/step - loss: 0.1970 - mse: 0.1970
Epoch 76/150
94/94 [==============================] - 0s 599us/step - loss: 0.1969 - mse: 0.1969
Epoch 77/150
94/94 [==============================] - 0s 612us/step - loss: 0.1969 - mse: 0.1969
Epoch 78/150
94/94 [==============================] - 0s 575us/step - loss: 0.1967 - mse: 0.1967
Epoch 79/150
94/94 [==============================] - 0s 609us/step - loss: 0.1965 - mse: 0.1965
Epoch 80/150
94/94 [==============================] - 0s 632us/step - loss: 0.1967 - mse: 0.1967
Epoch 81/150
94/94 [==============================] - 0s 578us/step - loss: 0.1964 - mse: 0.1964
Epoch 82/150
94/94 [==============================] - 0s 587us/step - loss: 0.1964 - mse: 0.1964
Epoch 83/150
94/94 [=======================

In [70]:
# RMSE on train data (scaled)
mean_squared_error(y_train_scaled, cv_bigger_model_preds, squared=False)

0.4444330604358337

## Regularizing the Model to Achieve Balance  

Now that you have a powerful model (albeit an overfit one), we can now increase the generalization of the model by using some of the regularization techniques we discussed. Some options you have to try include:  
* Adding dropout
* Adding L1/L2 regularization
* Altering the layer architecture (add or remove layers similar to above)  

This process will be constrained by time and resources. Be sure to test at least two different methodologies, such as dropout and L2 regularization. If you have the time, feel free to continue experimenting. 

In [74]:
# Define a function that returns a compiled Keras model 
def create_regularized_model():
        # Initialize model
    model = models.Sequential()
    
    model.add(layers.Dropout(0.3))
    # First hidden layer
    model.add(layers.Dense(100, activation='relu')) #, input_shape=(n_features,)))
    model.add(layers.Dropout(0.3))

    # Second hidden layer
    model.add(layers.Dense(50, activation='relu'))
    model.add(layers.Dropout(0.3))
    
    model.add(layers.Dense(25, activation='relu'))
    model.add(layers.Dropout(0.3))
    
    # Output layer
    model.add(layers.Dense(1, activation='linear'))

    # Compile the model
    model.compile(optimizer='SGD', 
                  loss='mse',  
                  metrics=['mse']) 
    
    return model

In [75]:
# Wrap the above function for use in cross-validation
keras_wrapper_3 = KerasRegressor(create_regularized_model, epochs=100, batch_size=256)

In [76]:
# ⏰ This cell may take several mintes to run
# Generate cross-validated predictions
np.random.seed(123)
cv_dropout_preds = cross_val_predict(keras_wrapper_3, X_train_all, y_train_scaled, cv=5)

Epoch 1/100

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

94/94 [==============================] - 0s 2ms/step - loss: 0.6769 - mse: 0.6769
Epoch 2/100
94/94 [==============================] - 0s 2ms/step - loss: 0.5011 - mse: 0.5011
Epoch 3/100
94/94 [==============================] - 0s 2ms/step - loss: 0.4689 - mse: 0.4689
Epoch 4/100
94/94 [==============================] - 0s 2ms/step - loss: 0.4394 - mse: 0.4394
Epoch 5/100
94/94 [==============================] - 0s 2ms/step - loss: 0.4350 - mse: 0.4350
Epoch 6/100
94/94 [===================

94/94 [==============================] - 0s 2ms/step - loss: 0.3184 - mse: 0.3184
Epoch 80/100
94/94 [==============================] - 0s 2ms/step - loss: 0.3124 - mse: 0.3124
Epoch 81/100
94/94 [==============================] - 0s 2ms/step - loss: 0.3143 - mse: 0.3143
Epoch 82/100
94/94 [==============================] - 0s 2ms/step - loss: 0.3173 - mse: 0.3173
Epoch 83/100
94/94 [==============================] - 0s 2ms/step - loss: 0.3147 - mse: 0.3147
Epoch 84/100
94/94 [==============================] - 0s 2ms/step - loss: 0.3239 - mse: 0.3239
Epoch 85/100
94/94 [==============================] - 0s 2ms/step - loss: 0.3099 - mse: 0.3099
Epoch 86/100
94/94 [==============================] - 0s 2ms/step - loss: 0.3149 - mse: 0.3149
Epoch 87/100
94/94 [==============================] - 0s 2ms/step - loss: 0.3132 - mse: 0.3132
Epoch 88/100
94/94 [==============================] - 0s 2ms/step - loss: 0.3142 - mse: 0.3142
Epoch 89/100
94/94 [==============================] - 0s 2ms/st

Epoch 58/100
94/94 [==============================] - 0s 2ms/step - loss: 0.3242 - mse: 0.3242
Epoch 59/100
94/94 [==============================] - 0s 2ms/step - loss: 0.3348 - mse: 0.3348
Epoch 60/100
94/94 [==============================] - 0s 2ms/step - loss: 0.3216 - mse: 0.3216
Epoch 61/100
94/94 [==============================] - 0s 2ms/step - loss: 0.3234 - mse: 0.3234
Epoch 62/100
94/94 [==============================] - 0s 2ms/step - loss: 0.3265 - mse: 0.3265
Epoch 63/100
94/94 [==============================] - 0s 2ms/step - loss: 0.3276 - mse: 0.3276
Epoch 64/100
94/94 [==============================] - 0s 2ms/step - loss: 0.3203 - mse: 0.3203
Epoch 65/100
94/94 [==============================] - 0s 2ms/step - loss: 0.3237 - mse: 0.3237
Epoch 66/100
94/94 [==============================] - 0s 2ms/step - loss: 0.3284 - mse: 0.3284
Epoch 67/100
94/94 [==============================] - 0s 2ms/step - loss: 0.3224 - mse: 0.3224
Epoch 68/100
94/94 [==============================

94/94 [==============================] - 0s 2ms/step - loss: 0.3411 - mse: 0.3411
Epoch 37/100
94/94 [==============================] - 0s 2ms/step - loss: 0.3430 - mse: 0.3430
Epoch 38/100
94/94 [==============================] - 0s 2ms/step - loss: 0.3470 - mse: 0.3470
Epoch 39/100
94/94 [==============================] - 0s 2ms/step - loss: 0.3351 - mse: 0.3351
Epoch 40/100
94/94 [==============================] - 0s 2ms/step - loss: 0.3361 - mse: 0.3361
Epoch 41/100
94/94 [==============================] - 0s 2ms/step - loss: 0.3358 - mse: 0.3358
Epoch 42/100
94/94 [==============================] - 0s 2ms/step - loss: 0.3344 - mse: 0.3344
Epoch 43/100
94/94 [==============================] - 0s 2ms/step - loss: 0.3395 - mse: 0.3395
Epoch 44/100
94/94 [==============================] - 0s 2ms/step - loss: 0.3306 - mse: 0.3306
Epoch 45/100
94/94 [==============================] - 0s 2ms/step - loss: 0.3305 - mse: 0.3305
Epoch 46/100
94/94 [==============================] - 0s 2ms/st

Epoch 15/100
94/94 [==============================] - 0s 2ms/step - loss: 0.3740 - mse: 0.3740
Epoch 16/100
94/94 [==============================] - 0s 2ms/step - loss: 0.3670 - mse: 0.3670
Epoch 17/100
94/94 [==============================] - 0s 2ms/step - loss: 0.3672 - mse: 0.3672
Epoch 18/100
94/94 [==============================] - 0s 2ms/step - loss: 0.3568 - mse: 0.3568
Epoch 19/100
94/94 [==============================] - 0s 2ms/step - loss: 0.3683 - mse: 0.3683
Epoch 20/100
94/94 [==============================] - 0s 2ms/step - loss: 0.3609 - mse: 0.3609
Epoch 21/100
94/94 [==============================] - 0s 2ms/step - loss: 0.3602 - mse: 0.3602
Epoch 22/100
94/94 [==============================] - 0s 2ms/step - loss: 0.3553 - mse: 0.3553
Epoch 23/100
94/94 [==============================] - 0s 2ms/step - loss: 0.3502 - mse: 0.3502
Epoch 24/100
94/94 [==============================] - 0s 2ms/step - loss: 0.3544 - mse: 0.3544
Epoch 25/100
94/94 [==============================

94/94 [==============================] - 0s 2ms/step - loss: 0.6633 - mse: 0.6633
Epoch 2/100
94/94 [==============================] - 0s 2ms/step - loss: 0.5140 - mse: 0.5140
Epoch 3/100
94/94 [==============================] - 0s 2ms/step - loss: 0.4675 - mse: 0.4675
Epoch 4/100
94/94 [==============================] - 0s 2ms/step - loss: 0.4440 - mse: 0.4440
Epoch 5/100
94/94 [==============================] - 0s 2ms/step - loss: 0.4254 - mse: 0.4254
Epoch 6/100
94/94 [==============================] - 0s 2ms/step - loss: 0.4182 - mse: 0.4182
Epoch 7/100
94/94 [==============================] - 0s 2ms/step - loss: 0.4246 - mse: 0.4246
Epoch 8/100
94/94 [==============================] - 0s 2ms/step - loss: 0.4033 - mse: 0.4033
Epoch 9/100
94/94 [==============================] - 0s 2ms/step - loss: 0.3943 - mse: 0.3943
Epoch 10/100
94/94 [==============================] - 0s 2ms/step - loss: 0.3963 - mse: 0.3963
Epoch 11/100
94/94 [==============================] - 0s 2ms/step - los

94/94 [==============================] - 0s 2ms/step - loss: 0.3264 - mse: 0.3264
Epoch 88/100
94/94 [==============================] - 0s 2ms/step - loss: 0.3156 - mse: 0.3156
Epoch 89/100
94/94 [==============================] - 0s 2ms/step - loss: 0.3205 - mse: 0.3205
Epoch 90/100
94/94 [==============================] - 0s 2ms/step - loss: 0.3119 - mse: 0.3119
Epoch 91/100
94/94 [==============================] - 0s 2ms/step - loss: 0.3138 - mse: 0.3138
Epoch 92/100
94/94 [==============================] - 0s 2ms/step - loss: 0.3155 - mse: 0.3155
Epoch 93/100
94/94 [==============================] - 0s 2ms/step - loss: 0.3198 - mse: 0.3198
Epoch 94/100
94/94 [==============================] - 0s 2ms/step - loss: 0.3067 - mse: 0.3067
Epoch 95/100
94/94 [==============================] - 0s 2ms/step - loss: 0.3142 - mse: 0.3142
Epoch 96/100
94/94 [==============================] - 0s 2ms/step - loss: 0.3178 - mse: 0.3178
Epoch 97/100
94/94 [==============================] - 0s 2ms/st

In [77]:
# RMSE on train data (scaled)
mean_squared_error(y_train_scaled, cv_dropout_preds, squared=False)

0.5900240191190366

## Final Evaluation

Now that you have selected a network architecture, tested various regularization procedures and tuned hyperparameters via a validation methodology, it is time to evaluate your final model on the test set. Fit the model using all of the training data using the architecture and hyperparameters that were most effective in your experiments above. Afterwards, measure the overall performance on the hold-out test data which has been left untouched (and hasn't leaked any data into the modeling process)! 

In [80]:
# ⏰ This cell may take several mintes to run
model = create_regularized_model()
model_val = model.fit(X_train_all, y_train_scaled, epochs=100, batch_size=256)

Epoch 1/100

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

117/117 [==============================] - 0s 2ms/step - loss: 0.6168 - mse: 0.6168
Epoch 2/100
117/117 [==============================] - 0s 2ms/step - loss: 0.4936 - mse: 0.4936
Epoch 3/100
117/117 [==============================] - 0s 2ms/step - loss: 0.4748 - mse: 0.4748
Epoch 4/100
117/117 [==============================] - 0s 2ms/step - loss: 0.4294 - mse: 0.4294
Epoch 5/100
117/117 [==============================] - 0s 2ms/step - loss: 0.4097 - mse: 0.4097
Epoch 6/100
117/117 [=======

117/117 [==============================] - 0s 2ms/step - loss: 0.3118 - mse: 0.3118
Epoch 79/100
117/117 [==============================] - 0s 2ms/step - loss: 0.3141 - mse: 0.3141
Epoch 80/100
117/117 [==============================] - 0s 2ms/step - loss: 0.3128 - mse: 0.3128
Epoch 81/100
117/117 [==============================] - 0s 2ms/step - loss: 0.3152 - mse: 0.3152
Epoch 82/100
117/117 [==============================] - 0s 2ms/step - loss: 0.3119 - mse: 0.3119
Epoch 83/100
117/117 [==============================] - 0s 2ms/step - loss: 0.3103 - mse: 0.3103
Epoch 84/100
117/117 [==============================] - 0s 2ms/step - loss: 0.3100 - mse: 0.3100
Epoch 85/100
117/117 [==============================] - 0s 2ms/step - loss: 0.3086 - mse: 0.3086
Epoch 86/100
117/117 [==============================] - 0s 2ms/step - loss: 0.3126 - mse: 0.3126
Epoch 87/100
117/117 [==============================] - 0s 2ms/step - loss: 0.3045 - mse: 0.3045
Epoch 88/100
117/117 [=====================

In [84]:
# change to original units
y_pred_scaled = model.predict(X_test_all)
y_pred = ss_y.inverse_transform(y_pred_scaled)
mean_squared_error(y_test_scaled, y_pred, squared=False)

12314.250023537952

## Summary

In this lab, you investigated some data from *The Lending Club* in a complete data science pipeline to build neural networks with good performance. You began with reserving a hold-out set for testing which never was touched during the modeling phase. From there, you implemented a k-fold cross validation methodology in order to assess an initial baseline model and various regularization methods. 